In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("oscaryezfeijo/us-congressional-tweets-dataset")

print("Path to dataset files:", path)

/Users/christianhjorth/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


100%|██████████| 232M/232M [00:14<00:00, 17.1MB/s] 

Extracting files...


Path to dataset files: /Users/christianhjorth/.cache/kagglehub/datasets/oscaryezfeijo/us-congressional-tweets-dataset/versions/1


In [2]:
!pip install kagglehub

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 5.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [4]:
import pandas as pd

/var/folders/bm/yntdr4q97s74g63m6p8162rw0000gn/T/ipykernel_78920/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
import pandas as pd

# Reads the JSON file into a DataFrame
df = pd.read_json('tweets.json', lines=True)

# Displays the DataFrame
print(df)

In [23]:
import pandas as pd
import json
from tqdm import tqdm

def parse_tweets(file_path, fields):
    def generate_filtered_rows():
        with open(file_path, 'r', encoding='utf-8') as f:
            total_lines = sum(1 for line in f)  # Get the total number of lines (tweets)
            f.seek(0)  # Reset file pointer after counting lines
            for line in tqdm(f, total=total_lines, desc="Processing Tweets"):
                try:
                    tweet = json.loads(line)
                    yield {field: tweet.get(field, None) for field in fields}
                except json.JSONDecodeError:
                    continue  # Skip malformed lines
    return pd.DataFrame(generate_filtered_rows())

# Define the fields
fields_to_extract = ["created_at", "id", "retweeted", "screen_name", "text", "user_id"]

# Load into DataFrame
tweets1 = parse_tweets("tweets.json", fields_to_extract)

# Optional
# df.to_csv("output_file.csv", index=False)

Processing Tweets: 100%|██████████| 1243370/1243370 [00:19<00:00, 64090.75it/s]


In [24]:
import re

# Remove all http(s) URLs from text
tweets1["text"] = tweets1["text"].str.replace(r"http\S+|www\.\S+", "", regex=True)

# Also strip leftover whitespace
tweets1["text"] = tweets1["text"].str.strip()

In [25]:
tweets1["text"] = tweets1["text"].str.replace(r"\n", "\\n")

In [28]:
tweets1["date"] = pd.to_datetime(tweets1["created_at"], unit="s").dt.date.astype(str)

In [30]:
tweets1.drop(columns=["created_at"], inplace=True, errors="ignore")


In [45]:
print(tweets1["retweeted"].unique())

[False]


In [46]:
tweets1.drop(columns=["retweeted"], inplace=True)

In [50]:
tweets1 = tweets1[tweets1["date"] >= "2011-01-01"]

In [48]:
tweets1 = tweets1[["id", "screen_name", "user_id", "text", "date"]]

In [4]:
import requests
import os

# Folder to save the downloaded JSON files
output_dir = "tweets2"
os.makedirs(output_dir, exist_ok=True)

# GitHub API URL for the folder contents
api_url = "https://api.github.com/repos/alexlitel/congresstweets/contents/data"

# Get list of files in the data/ directory
response = requests.get(api_url)
response.raise_for_status()
files = response.json()

# Download each JSON file
for file in files:
    filename = file['name']
    download_url = file['download_url']

    if filename.endswith(".json"):
        print(f"Downloading: {filename}")
        content = requests.get(download_url)
        content.raise_for_status()

        # Save the file locally
        with open(os.path.join(output_dir, filename), 'wb') as f:
            f.write(content.content)

/Users/christianhjorth/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Downloading: 2017-06-21.json
Downloading: 2017-06-22.json
Downloading: 2017-06-23.json
Downloading: 2017-06-24.json
Downloading: 2017-06-25.json
Downloading: 2017-06-26.json
Downloading: 2017-06-27.json
Downloading: 2017-06-28.json
Downloading: 2017-06-29.json
Downloading: 2017-06-30.json
Downloading: 2017-07-01.json
Downloading: 2017-07-02.json
Downloading: 2017-07-03.json
Downloading: 2017-07-04.json
Downloading: 2017-07-05.json
Downloading: 2017-07-06.json
Downloading: 2017-07-07.json
Downloading: 2017-07-08.json
Downloading: 2017-07-09.json
Downloading: 2017-07-10.json
Downloading: 2017-07-11.json
Downloading: 2017-07-12.json
Downloading: 2017-07-13.json
Downloading: 2017-07-14.json
Downloading: 2017-07-15.json
Downloading: 2017-07-16.json
Downloading: 2017-07-17.json
Downloading: 2017-07-18.json
Downloading: 2017-07-19.json
Downloading: 2017-07-20.json
Downloading: 2017-07-21.json
Downloading: 2017-07-22.json
Downloading: 2017-07-23.json
Downloading: 2017-07-24.json
Downloading: 2

In [10]:
import os
import requests
from datetime import datetime, timedelta

# Config
output_dir = "tweets2"
start_date = datetime.strptime("2020-03-21", "%Y-%m-%d")
end_date = datetime.strptime("2023-07-11", "%Y-%m-%d")
base_url = "https://raw.githubusercontent.com/alexlitel/congresstweets/master/data"
os.makedirs(output_dir, exist_ok=True)

# Loop over each date and download the corresponding JSON file
date = start_date
while date <= end_date:
    filename = f"{date.strftime('%Y-%m-%d')}.json"
    url = f"{base_url}/{filename}"
    out_path = os.path.join(output_dir, filename)

    if os.path.exists(out_path):
        print(f"Already downloaded: {filename}")
        date += timedelta(days=1)
        continue

    print(f"Downloading: {filename}")
    response = requests.get(url)
    if response.status_code == 200:
        with open(out_path, "wb") as f:
            f.write(response.content)
    else:
        print(f"Failed to download {filename} (status code {response.status_code})")

    date += timedelta(days=1)

Downloading: 2020-03-21.json
Downloading: 2020-03-22.json
Downloading: 2020-03-23.json
Downloading: 2020-03-24.json
Downloading: 2020-03-25.json
Downloading: 2020-03-26.json
Downloading: 2020-03-27.json
Downloading: 2020-03-28.json
Downloading: 2020-03-29.json
Downloading: 2020-03-30.json
Downloading: 2020-03-31.json
Downloading: 2020-04-01.json
Downloading: 2020-04-02.json
Downloading: 2020-04-03.json
Downloading: 2020-04-04.json
Downloading: 2020-04-05.json
Downloading: 2020-04-06.json
Downloading: 2020-04-07.json
Downloading: 2020-04-08.json
Downloading: 2020-04-09.json
Downloading: 2020-04-10.json
Downloading: 2020-04-11.json
Downloading: 2020-04-12.json
Downloading: 2020-04-13.json
Downloading: 2020-04-14.json
Downloading: 2020-04-15.json
Downloading: 2020-04-16.json
Downloading: 2020-04-17.json
Downloading: 2020-04-18.json
Downloading: 2020-04-19.json
Downloading: 2020-04-20.json
Downloading: 2020-04-21.json
Downloading: 2020-04-22.json
Downloading: 2020-04-23.json
Downloading: 2

In [2]:
import os
import json
import pandas as pd

folder_path = 'tweets2'
tweets_data = []

# Loop through all JSON files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            try:
                tweets = json.load(f)  # Each file is a list of tweet dicts
                for tweet in tweets:
                    tweets_data.append({
                        'id': tweet.get('id'),
                        'screen_name': tweet.get('screen_name'),
                        'user_id': tweet.get('user_id'),
                        'time': tweet.get('time'),
                        'text': tweet.get('text')
                    })
            except Exception as e:
                print(f"Error parsing {filename}: {e}")

# Convert to DataFrame
tweets2 = pd.DataFrame(tweets_data)

# Optional: parse 'time' as datetime
# df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Preview
print(tweets2.head())

/var/folders/bm/yntdr4q97s74g63m6p8162rw0000gn/T/ipykernel_17538/677162210.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


                    id     screen_name             user_id  \
0  1197531273473994755   RepValDemings  798973032362606600   
1  1197574591130013700    SenatorLeahy           242836537   
2  1197605237969866752      RepJayapal  815733290955112448   
3  1197611392427286540    JackieSpeier           521731529   
4  1197706911375863808  SenCoryGardner           235217558   

                        time  \
0  2019-11-21T10:04:39-05:00   
1  2019-11-21T12:56:47-05:00   
2  2019-11-21T14:58:33-05:00   
3  2019-11-21T15:23:01-05:00   
4  2019-11-21T21:42:34-05:00   

                                                text  
0  President Trump demanded that Ukraine interfer...  
1  Yes. He was the soul and patron saint of snowb...  
2  This violence is predictable and preventable, ...  
3  Last week, my legislation removing the deadlin...  
4  Now they want to do the same with corporations...  


In [5]:
# print(tweets2["text"].iloc[0])
print(tweets2.shape[0])
tweets2.head()

5068352


,id,screen_name,user_id,time,text
0,1197531273473994755,RepValDemings,798973032362606600,2019-11-21T10:04:39-05:00,President Trump demanded that Ukraine interfer...
1,1197574591130013700,SenatorLeahy,242836537,2019-11-21T12:56:47-05:00,Yes. He was the soul and patron saint of snowb...
2,1197605237969866752,RepJayapal,815733290955112448,2019-11-21T14:58:33-05:00,"This violence is predictable and preventable, ..."
3,1197611392427286540,JackieSpeier,521731529,2019-11-21T15:23:01-05:00,"Last week, my legislation removing the deadlin..."
4,1197706911375863808,SenCoryGardner,235217558,2019-11-21T21:42:34-05:00,Now they want to do the same with corporations...


In [6]:
import re

# Remove all http(s) URLs from text
tweets2["text"] = tweets2["text"].str.replace(r"http\S+|www\.\S+", "", regex=True)

# Also strip leftover whitespace
tweets2["text"] = tweets2["text"].str.strip()

In [8]:
tweets2["text"] = tweets2["text"].str.replace(r"\n", "\\n")

In [36]:
# Force conversion from string to datetime, handling offset properly
tweets2["time"] = pd.to_datetime(tweets2["time"], utc=True)

# Now you can safely access .dt
tweets2["date"] = tweets2["time"].dt.date.astype(str)

In [38]:
tweets2.drop(columns=["time"], inplace=True, errors="ignore")

In [51]:
tweets1

,id,screen_name,user_id,text,date
22109,20992897130176513,clairecmc,16160352,@ceruleanbill Doesn't mean you're not brillian...,2011-01-01
22110,21014756991172608,clairecmc,16160352,Big dangerous storm hit my sister's neighborho...,2011-01-01
22111,21054995990323200,clairecmc,16160352,@jeffmw Thanks Jeff. Happy New Year! Here's t...,2011-01-01
22112,21057954908213248,clairecmc,16160352,"5,4,3,2,1..happy new year! I know I'm early, b...",2011-01-01
22113,21069453831700481,RepJohnLarson,50452197,Wishing you and yours a Happy New Year!,2011-01-01
...,...,...,...,...,...
1243365,872139777444917249,PramilaJayapal,193441812,Dismantling #DoddFrank returns us to the days ...,2017-06-06
1243366,872139786378780677,RepSarbanes,364415553,"In the shadows of the #ComeyHearing, @HouseGOP...",2017-06-06
1243367,872139844717338624,SenMarkey,21406834,@BetsyDeVosED How does a budget that cuts inve...,2017-06-06
1243368,872140013416443906,MarioDB,37094727,Thank you @POTUS @NikkiHaley for strong stance...,2017-06-06


In [52]:
tweets2

,id,screen_name,user_id,text,date
0,1197531273473994755,RepValDemings,798973032362606600,President Trump demanded that Ukraine interfer...,2019-11-21
1,1197574591130013700,SenatorLeahy,242836537,Yes. He was the soul and patron saint of snowb...,2019-11-21
2,1197605237969866752,RepJayapal,815733290955112448,"This violence is predictable and preventable, ...",2019-11-21
3,1197611392427286540,JackieSpeier,521731529,"Last week, my legislation removing the deadlin...",2019-11-21
4,1197706911375863808,SenCoryGardner,235217558,Now they want to do the same with corporations...,2019-11-22
...,...,...,...,...,...
5068347,1057305960144928768,SmallBizCmte,2932401927,RT @SenateGOP Women are seeing the benefits of...,2018-10-30
5068348,1057265725323333633,TeamHydeSmith,1006217719002357767,RT @CityofGulfport Full house for @SenHydeSmit...,2018-10-30
5068349,1057382236855693312,PingreeForME,1002275817227980800,Looking forward to our third and final debate ...,2018-10-30
5068350,1057290542067765248,PingreeForME,1002275817227980800,"We can disagree about policy, but violence - a...",2018-10-30


In [56]:
# Merge the two datasets
tweets_combined = pd.concat([tweets1, tweets2], ignore_index=True)

# Optional: drop any duplicate tweets by ID
tweets_combined.drop_duplicates(subset="id", inplace=True)

# Save as gzip-compressed CSV
tweets_combined.to_csv("tweets_cleaned.csv.gz", index=False, compression="gzip", quoting=1)

In [18]:
import json
import pandas as pd

# Load the JSON list
with open("historical-users-filtered.json", "r") as f:
    data = json.load(f)

# Extract relevant fields
records = []
for entry in data:
    name = entry.get("name", "")
    for account in entry.get("accounts", []):
        records.append({
            "name": name,
            "id": account.get("id"),
            "screen_name": account.get("screen_name"),
            "account_type": account.get("account_type")
        })

# Convert to DataFrame
df_hist_users1 = pd.DataFrame(records)

# Preview
print(df_hist_users1.head())

                                          name                  id  \
0                                 ASEAN Caucus  899998766845100032   
1                           Blue Collar Caucus  817097052375187457   
2                           Blue Dog Coalition           224685124   
3               Congressional Antitrust Caucus  951207213728698374   
4  Congressional Asian Pacific American Caucus           192955168   

       screen_name account_type  
0      ASEANCaucus       office  
1   BlueCollarDems       office  
2    HouseBlueDogs       office  
3  AntitrustCaucus       office  
4            CAPAC       office  


In [21]:
df_hist_users1

,name,id,screen_name,account_type
0,ASEAN Caucus,899998766845100032,ASEANCaucus,office
1,Blue Collar Caucus,817097052375187457,BlueCollarDems,office
2,Blue Dog Coalition,224685124,HouseBlueDogs,office
3,Congressional Antitrust Caucus,951207213728698374,AntitrustCaucus,office
4,Congressional Asian Pacific American Caucus,192955168,CAPAC,office
...,...,...,...,...
1607,Senate Democrats,1148973240355827713,SenDemsClimate,office
1608,Senate Republicans,5693842,NRSC,campaign
1609,Senate Republicans,14344823,SenateGOP,office
1610,Senate Republicans,114896223,SenateRPC,office


In [22]:
import json
import pandas as pd

# Path to your users.json file
path = "users.json"

records = []

# Read line-by-line to handle large files efficiently
with open(path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            user = json.loads(line)
            records.append({
                "name": user.get("name"),
                "id": user.get("id"),
                "screen_name": user.get("screen_name"),
                "account_type": "unknown"
            })
        except json.JSONDecodeError:
            continue  # Skip malformed lines

# Create DataFrame
df_users_json = pd.DataFrame(records)

# Preview
print(df_users_json.head())

                   name          id      screen_name account_type
0  Governor Bill Walker  2915095729  AkGovBillWalker      unknown
1         Amy Klobuchar    33537967     amyklobuchar      unknown
2      Anthony G. Brown  1378000346  AnthonyBrownMD4      unknown
3   Gov. Asa Hutchinson   269992801    AsaHutchinson      unknown
4     Rep. Austin Scott   234797704  AustinScottGA08      unknown


In [23]:
df_users_json

,name,id,screen_name,account_type
0,Governor Bill Walker,2915095729,AkGovBillWalker,unknown
1,Amy Klobuchar,33537967,amyklobuchar,unknown
2,Anthony G. Brown,1378000346,AnthonyBrownMD4,unknown
3,Gov. Asa Hutchinson,269992801,AsaHutchinson,unknown
4,Rep. Austin Scott,234797704,AustinScottGA08,unknown
...,...,...,...,...
543,Virginia Foxx,16256269,virginiafoxx,unknown
544,Warren Davidson,742735530287304704,WarrenDavidson,unknown
545,Steny Hoyer,22012091,WhipHoyer,unknown
546,Governor Jim Justice,607332953,WVGovernor,unknown


In [25]:
# Combine the two DataFrames
combined_df = pd.concat([df_hist_users1, df_users_json], ignore_index=True)

# Optional: drop duplicate Twitter accounts (by ID or screen_name)
combined_df = combined_df.drop_duplicates(subset=["id"])

# Save as CSV
combined_df.to_csv("combined_twitter_users.csv", index=False)